In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# from tensorflow.python.framework import ops
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [5]:
mnist = input_data.read_data_sets("G:/MNIST_data/", one_hot = True)
logdir = 'G:/MNIST_data/'
if not os.path.exists(logdir):
    os.makedirs(logdir)

layer_dims = [784, 1024, 1024, 2048,10]  # 多层网络
L = len(layer_dims)  # L = 5
learning_rate = 0.01
batch_size = 64
total_batch = int(mnist.train.num_examples/batch_size)
num_epochs = 100

# parameters = {}
# for l in range(1,L):
#     parameters['W'+str(l)] = tf.get_variable('W'+str(l), [layer_dims[l], layer_dims[l-1],
#                                                           initializer = tf.contrib.layers.ariance_scaling_initializer()])
#     parameters['b'+str(l)] = tf.get.variable('b'+str(l), [layer_dims[l], 1], initializer = tf.zeros_initializer())
tf.reset_default_graph()
def fc_layer(n_th,  # 层序号
             inputs,  # 输入数据
             input_dims,  # 前一层节点数
             output_dims,  # 当前层节点数
             activation = None):  # 激活函数选项
    
    W = tf.get_variable('W' + str(n_th), [input_dims,output_dims], initializer = tf.contrib.layers.variance_scaling_initializer())
    b = tf.get_variable('b' + str(n_th), [1,output_dims], initializer = tf.zeros_initializer())
    Z = tf.add(tf.matmul(inputs, W), b)
    tf.summary.histogram('W'+str(n_th), W)
    tf.summary.histogram('b'+str(n_th), b)
    
    if activation == None:
        outputs = Z
    elif activation == 'relu':
        outputs = tf.nn.relu(Z)
        
    return outputs

with tf.name_scope('input_data'):
    X = tf.placeholder(tf.float32, [None, layer_dims[0]], name = 'X')
    Y = tf.placeholder(tf.float32, [None, layer_dims[L-1]], name = 'Y')
    image_shaped_input = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('X', image_shaped_input, 10)

A = X
for l in range(1, L-1):
    with tf.name_scope('fc_layer'+str(l)):
        A_ = A
        A = fc_layer(l, A_, layer_dims[l-1], layer_dims[l], activation = 'relu')
        A = tf.nn.dropout(A, keep_prob=0.8)
        tf.summary.histogram('A'+str(l), A)

with tf.name_scope('output_layer'):
    ZL = fc_layer(L-1, A, layer_dims[L-2], layer_dims[L-1], activation = None)
#     tf.summary.histogram('ZL', ZL)
    AL = tf.nn.softmax(ZL)
    tf.summary.histogram('A'+str(L-1), AL)
    
with tf.name_scope('loss'):
#     Y = tf.placeholder(tf.float32, [layer_dims[L-1], None], name = 'Y')
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = ZL, labels = Y))
    tf.summary.scalar('mean', loss)
    
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(ZL,1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    tf.summary.scalar('tr', accuracy)
        

Extracting G:/MNIST_data/train-images-idx3-ubyte.gz
Extracting G:/MNIST_data/train-labels-idx1-ubyte.gz
Extracting G:/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting G:/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(logdir, sess.graph)
    saver = tf.train.Saver()
    sess.run(init)
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch in range(total_batch):
            mini_X, mini_Y = mnist.train.next_batch(batch_size)
#             mini_X, mini_Y = mini_X.T, mini_Y.T
            
            _, temp_loss, summary = sess.run([optimizer, loss, merged], feed_dict={X:mini_X, Y:mini_Y})
               
            writer.add_summary(summary, epoch)
            epoch_loss += temp_loss / total_batch 
            
        val_loss, val_acc = sess.run([loss, accuracy], feed_dict={X:mnist.validation.images, Y:mnist.validation.labels})
        
        if epoch % 5 == 0:
            print("Train epoch: {:>}, Train_loss = {:.6f} || Validation_loss = {:.6f}, Accuracy = {:.4f}".format(epoch, epoch_loss, val_loss, val_acc))
            saver.save(sess, os.path.join(logdir, 'mnist_fc_model.ckpt-{:05d}'.format(epoch)))
    
    saver.save(sess, os.path.join(logdir, 'mnist_fc_model.ckpt-{:05d}'.format(num_epochs)))
    print("Model saved!")
    

Train epoch: 0, Train_loss = 0.728191 || Validation_loss = 0.540119, Accuracy = 0.8624
Train epoch: 5, Train_loss = 0.478379 || Validation_loss = 0.465318, Accuracy = 0.8914
Train epoch: 10, Train_loss = 0.471081 || Validation_loss = 0.483859, Accuracy = 0.8726
Train epoch: 15, Train_loss = 0.443272 || Validation_loss = 0.524459, Accuracy = 0.8872
Train epoch: 20, Train_loss = 0.450570 || Validation_loss = 0.535776, Accuracy = 0.8982
Train epoch: 25, Train_loss = 0.457528 || Validation_loss = 0.572600, Accuracy = 0.8906
Train epoch: 30, Train_loss = 0.437739 || Validation_loss = 0.839381, Accuracy = 0.8830
Train epoch: 35, Train_loss = 0.460099 || Validation_loss = 0.544605, Accuracy = 0.8832
Train epoch: 40, Train_loss = 0.437522 || Validation_loss = 0.721794, Accuracy = 0.8856
Train epoch: 45, Train_loss = 0.476319 || Validation_loss = 0.797456, Accuracy = 0.8776
Train epoch: 50, Train_loss = 0.460576 || Validation_loss = 0.755249, Accuracy = 0.8844
Train epoch: 55, Train_loss = 0.44

In [9]:
mnist = input_data.read_data_sets("G:/MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting G:/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting G:/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting G:/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting G:/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
with tf.Session() as sess:
    graph = tf.get_default_graph()
    ckpt = tf.train.get_checkpoint_state('G:/MNIST_data/')
    
    if ckpt and ckpt.model_checkpoint_path:
        saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path +'.meta')
        saver.restore(sess, ckpt.model_checkpoint_path) # 从已保存模型中读取参数
        print("Restore model from ", ckpt.model_checkpoint_path)
        
    X = graph.get_tensor_by_name('input_data/X:0')
    loss = graph.get_tensor_by_name('loss/Mean:0')
    Y = graph.get_tensor_by_name('input_data/Y:0')
#     accuracy = graph.get_tensor_by_name('accuracy/tr:0')
    print(sess.run(loss, feed_dict={X: mnist.test.images,Y:mnist.test.labels}))
#     W1 = graph.get_tensor_by_name('W1:0')
#     print(sess.run(W1))      # print(sess.run('W1:0'))
    
#     for tensor in graph.as_graph_def().node:
#         print(tensor.name)
#     print(sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

INFO:tensorflow:Restoring parameters from G:/MNIST_data/mnist_fc_model.ckpt-00100
Restore model from  G:/MNIST_data/mnist_fc_model.ckpt-00100


In [5]:
# 统计错误预测
with tf.Session() as sess:
    prediction_result = sess.run(tf.argmax(ZL, 1), feed_dict={X: mnist.test.images})
def print_predict_errs(labels, prediction):
    count = 0
    compare_lists = (prediction==np.argmax(labels,1))
    err_lists = [i for i in range(len(compare_lists)) if compare_lists[1]==False]
    for x in err_lists:
        print('index=' + str(x) + ' 标签值（真实值）=', np.argmax(labels[x]), ' 预测值=', prediction[x])
        count += 1
    print('总计错误预测数:', count)
    
print_predic_errs(mnist.test.labels, prediction_result)

FailedPreconditionError: Attempting to use uninitialized value W1
	 [[Node: W1/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](W1)]]
	 [[Node: ArgMax/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_71_ArgMax", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'W1/read', defined at:
  File "E:\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "E:\anaconda3\envs\tensorflow-gpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    ap

In [ ]:
# 可视化函数
def plot_images_labels_prediction(images, # 图像列表
                                  labels, # 标签
                                  prediction, # 预测值
                                  index=0, # 从第index开始 
                                  num=10) # 一次显示
    fig = plt.gcf()
    fig.set_size_inches(10, 12)
    if num > 25:
        num = 25
    for in range(num):
        ax = plt.subplot(5,5,i+1)
        ax.imshow(np.reshape(images[index], (28,28)), cmap = 'binary')
        title = 'label=' + str(np.argmax(labels[index]))
        if len(prediction>0):
            title += ', predic=' + str(prediction[index])
        
        ax.set_title(title, fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
        index += 1
    plt.show()
        
plot_images_labels_prediction(mnist.test.images, mnist.test.labels, prediction_result, 0, 10)